# DSA Assignment (Practice Questions)  
**Submitter Name:** Aasif Majeed  
**Date:** 29 Dec 2023  

This notebook provides **complete solutions to all 32 problems** from the given assignment sheet.

**How to read this notebook**
- Each section starts with the **question (with problem number)**.
- Then you will see a short **explanation**.
- Then a **Python implementation** plus a **demo using the sample input/output** (where provided).


## Common Helpers (Used in Multiple Problems)

We implement:
- A **singly linked list node** (`ListNode`) + builders
- A **multilevel doubly linked list node** (`DNode`) for Problem 11
- Small helper functions to print/convert lists


In [1]:
# =========================
# Common Helpers
# =========================

from typing import Optional, List, Tuple
import heapq
import math

# ---------- Singly Linked List ----------
class ListNode:
    def __init__(self, val: int = 0, next: Optional["ListNode"] = None):
        self.val = val
        self.next = next

def build_list(arr: List[int]) -> Optional[ListNode]:
    head = None
    cur = None
    for x in arr:
        if head is None:
            head = cur = ListNode(x)
        else:
            cur.next = ListNode(x)
            cur = cur.next
    return head

def list_to_array(head: Optional[ListNode]) -> List[int]:
    out = []
    while head:
        out.append(head.val)
        head = head.next
    return out

def print_list(head: Optional[ListNode]) -> None:
    print(" -> ".join(map(str, list_to_array(head))) if head else "EMPTY")


# ---------- Multilevel Doubly Linked List (Problem 11) ----------
class DNode:
    def __init__(self, val: int):
        self.val = val
        self.prev: Optional["DNode"] = None
        self.next: Optional["DNode"] = None
        self.child: Optional["DNode"] = None

def doubly_to_array(head: Optional[DNode]) -> List[int]:
    out = []
    cur = head
    while cur:
        out.append(cur.val)
        cur = cur.next
    return out

def print_doubly(head: Optional[DNode]) -> None:
    arr = doubly_to_array(head)
    print(" <-> ".join(map(str, arr)) if arr else "EMPTY")


## Problem 1: Reverse a singly linked list

### Question
**Input:** `1 -> 2 -> 3 -> 4 -> 5`  
**Output:** `5 -> 4 -> 3 -> 2 -> 1`

### Explanation (Approach)
Use three pointers:
- `prev` initially `None`
- `cur` starting at head
- `nxt` to temporarily store `cur.next`

Reverse links one-by-one: `cur.next = prev`, then advance `prev` and `cur`.
Time: **O(n)**, Space: **O(1)**.


In [2]:
def reverse_list(head: Optional[ListNode]) -> Optional[ListNode]:
    prev = None
    cur = head
    while cur:
        nxt = cur.next
        cur.next = prev
        prev = cur
        cur = nxt
    return prev

# Demo
head = build_list([1, 2, 3, 4, 5])
print("Input : ", list_to_array(head))
out = reverse_list(head)
print("Output: ", list_to_array(out))


Input :  [1, 2, 3, 4, 5]
Output:  [5, 4, 3, 2, 1]


## Problem 2: Merge two sorted linked lists into one sorted linked list

### Question
**Input:** List 1: `1 -> 3 -> 5`, List 2: `2 -> 4 -> 6`  
**Output:** `1 -> 2 -> 3 -> 4 -> 5 -> 6`

### Explanation (Approach)
Use a dummy node and a tail pointer.
Repeatedly take the smaller head node from `l1` or `l2`, append to merged list, and advance.
Finally append the remaining nodes.
Time: **O(n+m)**, Space: **O(1)** extra (excluding output list links).


In [3]:
def merge_sorted_lists(l1: Optional[ListNode], l2: Optional[ListNode]) -> Optional[ListNode]:
    dummy = ListNode(0)
    tail = dummy
    while l1 and l2:
        if l1.val <= l2.val:
            tail.next = l1
            l1 = l1.next
        else:
            tail.next = l2
            l2 = l2.next
        tail = tail.next
    tail.next = l1 or l2
    return dummy.next

# Demo
l1 = build_list([1, 3, 5])
l2 = build_list([2, 4, 6])
merged = merge_sorted_lists(l1, l2)
print("Output:", list_to_array(merged))


Output: [1, 2, 3, 4, 5, 6]


## Problem 3: Remove the nth node from the end of a linked list

### Question
**Input:** `1 -> 2 -> 3 -> 4 -> 5`, `n = 2`  
**Output:** `1 -> 2 -> 3 -> 5`

### Explanation (Approach)
Use the two-pointer technique:
- Move `fast` pointer `n` steps ahead.
- Move `fast` and `slow` together until `fast` reaches the end.
- `slow.next` is the node to remove.

Use a dummy node so removing the head is easy.
Time: **O(n)**, Space: **O(1)**.


In [4]:
def remove_nth_from_end(head: Optional[ListNode], n: int) -> Optional[ListNode]:
    dummy = ListNode(0, head)
    fast = slow = dummy

    # move fast n steps
    for _ in range(n):
        fast = fast.next
        if fast is None:
            return head  # n larger than length

    # move both until fast at last node
    while fast.next:
        fast = fast.next
        slow = slow.next

    # delete slow.next
    slow.next = slow.next.next
    return dummy.next

# Demo
head = build_list([1, 2, 3, 4, 5])
out = remove_nth_from_end(head, 2)
print("Output:", list_to_array(out))


Output: [1, 2, 3, 5]


## Problem 4: Find the intersection point of two linked lists

### Question
**Input:** List 1: `1 -> 2 -> 3 -> 4`, List 2: `9 -> 8 -> 3 -> 4`  
**Output:** Node with value `3`

### Explanation (Approach)
Classic trick: pointer switching.
Let `a` start at headA and `b` start at headB.
Advance both by one; when one reaches the end, redirect it to the other head.
If lists intersect, pointers meet at intersection after at most `lenA + lenB` steps.
Time: **O(n+m)**, Space: **O(1)**.


In [5]:
def get_intersection_node(headA: Optional[ListNode], headB: Optional[ListNode]) -> Optional[ListNode]:
    a, b = headA, headB
    while a is not b:
        a = a.next if a else headB
        b = b.next if b else headA
    return a  # intersection node or None

# Demo build intersection:
# Common part: 3 -> 4
common = build_list([3, 4])
# ListA: 1 -> 2 -> (3 -> 4)
headA = build_list([1, 2])
tailA = headA
while tailA.next:
    tailA = tailA.next
tailA.next = common
# ListB: 9 -> 8 -> (3 -> 4)
headB = build_list([9, 8])
tailB = headB
while tailB.next:
    tailB = tailB.next
tailB.next = common

inter = get_intersection_node(headA, headB)
print("Intersection value:", inter.val if inter else None)


Intersection value: 3


## Problem 5: Remove duplicates from a sorted linked list

### Question
**Input:** `1 -> 1 -> 2 -> 3 -> 3`  
**Output:** `1 -> 2 -> 3`

### Explanation (Approach)
Because the list is sorted, duplicates are adjacent.
Traverse the list, and whenever `cur.val == cur.next.val`, skip `cur.next`.
Time: **O(n)**, Space: **O(1)**.


In [6]:
def delete_duplicates_sorted(head: Optional[ListNode]) -> Optional[ListNode]:
    cur = head
    while cur and cur.next:
        if cur.val == cur.next.val:
            cur.next = cur.next.next
        else:
            cur = cur.next
    return head

# Demo
head = build_list([1, 1, 2, 3, 3])
out = delete_duplicates_sorted(head)
print("Output:", list_to_array(out))


Output: [1, 2, 3]


## Problem 6: Add two numbers represented by linked lists (each node contains a single digit)

### Question
Each list stores digits in reverse order (least significant digit first).  
**Input:** List 1: `2 -> 4 -> 3`, List 2: `5 -> 6 -> 4` (represents 342 + 465)  
**Output:** `7 -> 0 -> 8` (represents 807)

### Explanation (Approach)
Simulate grade-school addition with carry:
- Sum current digits + carry
- New digit = sum % 10
- New carry = sum // 10
Continue until both lists and carry are exhausted.
Time: **O(n+m)**, Space: **O(max(n,m))** for output nodes.


In [7]:
def add_two_numbers(l1: Optional[ListNode], l2: Optional[ListNode]) -> Optional[ListNode]:
    dummy = ListNode(0)
    cur = dummy
    carry = 0

    while l1 or l2 or carry:
        x = l1.val if l1 else 0
        y = l2.val if l2 else 0
        s = x + y + carry
        carry = s // 10
        cur.next = ListNode(s % 10)
        cur = cur.next
        l1 = l1.next if l1 else None
        l2 = l2.next if l2 else None

    return dummy.next

# Demo
l1 = build_list([2, 4, 3])
l2 = build_list([5, 6, 4])
out = add_two_numbers(l1, l2)
print("Output:", list_to_array(out))


Output: [7, 0, 8]


## Problem 7: Swap nodes in pairs in a linked list

### Question
**Input:** `1 -> 2 -> 3 -> 4`  
**Output:** `2 -> 1 -> 4 -> 3`

### Explanation (Approach)
Use a dummy node and swap pairs by rewiring pointers:
Given `a -> b -> next`, we rewire to `b -> a -> next`.
Advance by two nodes each time.
Time: **O(n)**, Space: **O(1)**.


In [8]:
def swap_pairs(head: Optional[ListNode]) -> Optional[ListNode]:
    dummy = ListNode(0, head)
    prev = dummy

    while prev.next and prev.next.next:
        a = prev.next
        b = a.next

        # swap
        prev.next = b
        a.next = b.next
        b.next = a

        # advance
        prev = a

    return dummy.next

# Demo
head = build_list([1, 2, 3, 4])
out = swap_pairs(head)
print("Output:", list_to_array(out))


Output: [2, 1, 4, 3]


## Problem 8: Reverse nodes in a linked list in groups of k

### Question
**Input:** `1 -> 2 -> 3 -> 4 -> 5`, `k = 3`  
**Output:** `3 -> 2 -> 1 -> 4 -> 5`

### Explanation (Approach)
Reverse nodes in chunks of size `k`:
1. Check if at least `k` nodes remain.
2. Reverse exactly `k` nodes.
3. Connect previous chunk to reversed head.
If remaining nodes < k, leave as-is.
Time: **O(n)**, Space: **O(1)**.


In [9]:
def reverse_k_group(head: Optional[ListNode], k: int) -> Optional[ListNode]:
    if k <= 1 or not head:
        return head

    dummy = ListNode(0, head)
    group_prev = dummy

    while True:
        # find kth node
        kth = group_prev
        for _ in range(k):
            kth = kth.next
            if not kth:
                return dummy.next

        group_next = kth.next

        # reverse group
        prev, cur = group_next, group_prev.next
        while cur != group_next:
            nxt = cur.next
            cur.next = prev
            prev = cur
            cur = nxt

        # connect
        tmp = group_prev.next          # old head becomes tail
        group_prev.next = kth         # kth is new head
        group_prev = tmp

# Demo
head = build_list([1, 2, 3, 4, 5])
out = reverse_k_group(head, 3)
print("Output:", list_to_array(out))


Output: [3, 2, 1, 4, 5]


## Problem 9: Determine if a linked list is a palindrome

### Question
**Input:** `1 -> 2 -> 2 -> 1`  
**Output:** `True`

### Explanation (Approach)
Steps:
1. Find middle using slow/fast pointers.
2. Reverse the second half.
3. Compare first half and reversed second half.
(Optional) restore list after checking.
Time: **O(n)**, Space: **O(1)**.


In [10]:
def is_palindrome(head: Optional[ListNode]) -> bool:
    if not head or not head.next:
        return True

    # find mid
    slow = fast = head
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next

    # reverse second half
    prev = None
    cur = slow
    while cur:
        nxt = cur.next
        cur.next = prev
        prev = cur
        cur = nxt

    # compare
    left, right = head, prev
    while right:  # right half length <= left
        if left.val != right.val:
            return False
        left = left.next
        right = right.next
    return True

# Demo
head = build_list([1, 2, 2, 1])
print("Output:", is_palindrome(head))


Output: True


## Problem 10: Rotate a linked list to the right by k places

### Question
**Input:** `1 -> 2 -> 3 -> 4 -> 5`, `k = 2`  
**Output:** `4 -> 5 -> 1 -> 2 -> 3`

### Explanation (Approach)
Compute length `n`, reduce `k = k % n`.
Make list circular by connecting tail -> head.
New tail is at position `n-k-1` from head.
Break the circle to form the rotated list.
Time: **O(n)**, Space: **O(1)**.


In [11]:
def rotate_right(head: Optional[ListNode], k: int) -> Optional[ListNode]:
    if not head or not head.next or k == 0:
        return head

    # compute length and tail
    n = 1
    tail = head
    while tail.next:
        tail = tail.next
        n += 1

    k %= n
    if k == 0:
        return head

    # make circular
    tail.next = head

    # find new tail: n-k-1 steps
    steps = n - k - 1
    new_tail = head
    for _ in range(steps):
        new_tail = new_tail.next

    new_head = new_tail.next
    new_tail.next = None
    return new_head

# Demo
head = build_list([1, 2, 3, 4, 5])
out = rotate_right(head, 2)
print("Output:", list_to_array(out))


Output: [4, 5, 1, 2, 3]


## Problem 11: Flatten a multilevel doubly linked list

### Question
Flatten the list so that all nodes appear in a single-level doubly linked list (depth-first order).  
**Goal Output Example:** `1 <-> 2 <-> 3 <-> 4 <-> ...`

### Explanation (Approach)
Use an iterative stack-based DFS:
- Traverse nodes using `cur`.
- If `cur.child` exists:
  - Push `cur.next` to stack (to continue later)
  - Connect `cur` to `cur.child` as `cur.next`
  - Fix `.prev` pointers, set `cur.child = None`
- If reach end of current chain and stack not empty, pop and connect.

Time: **O(n)** where `n` is total nodes, Space: **O(depth)** for stack.


In [12]:
def flatten(head: Optional[DNode]) -> Optional[DNode]:
    if not head:
        return head

    stack = []
    cur = head

    while cur:
        if cur.child:
            if cur.next:
                stack.append(cur.next)
                cur.next.prev = None  # detach (optional safety)
            # connect child
            cur.next = cur.child
            cur.child.prev = cur
            cur.child = None

        if not cur.next and stack:
            nxt = stack.pop()
            cur.next = nxt
            nxt.prev = cur

        cur = cur.next

    return head

# Demo (simple multilevel structure)
# 1 <-> 2 <-> 3
#           |
#           4 <-> 5
one = DNode(1); two = DNode(2); three = DNode(3)
one.next = two; two.prev = one
two.next = three; three.prev = two
four = DNode(4); five = DNode(5)
four.next = five; five.prev = four
three.child = four

head = flatten(one)
print("Output:", doubly_to_array(head))


Output: [1, 2, 3, 4, 5]


## Problem 12: Rearrange a linked list such that all even positioned nodes are placed at the end

### Question
**Input:** `1 -> 2 -> 3 -> 4 -> 5`  
**Output:** `1 -> 3 -> 5 -> 2 -> 4`

### Explanation (Approach)
Maintain two lists:
- `odd` for positions 1,3,5,...
- `even` for positions 2,4,6,...
Then attach even list to the end of odd list.
Time: **O(n)**, Space: **O(1)**.


In [13]:
def odd_even_rearrange(head: Optional[ListNode]) -> Optional[ListNode]:
    if not head or not head.next:
        return head

    odd = head
    even = head.next
    even_head = even

    while even and even.next:
        odd.next = even.next
        odd = odd.next
        even.next = odd.next
        even = even.next

    odd.next = even_head
    return head

# Demo
head = build_list([1,2,3,4,5])
out = odd_even_rearrange(head)
print("Output:", list_to_array(out))


Output: [1, 3, 5, 2, 4]


## Problem 13: Given a non-negative number represented as a linked list, add one to it

### Question
Digits are in forward order.  
**Input:** `1 -> 2 -> 3` (represents 123)  
**Output:** `1 -> 2 -> 4` (represents 124)

### Explanation (Approach)
A simple approach:
1. Reverse list
2. Add 1 with carry
3. Reverse back

This avoids recursion depth issues and stays O(n).
Time: **O(n)**, Space: **O(1)** extra.


In [14]:
def add_one(head: Optional[ListNode]) -> Optional[ListNode]:
    if not head:
        return ListNode(1)

    # reverse
    head = reverse_list(head)

    cur = head
    carry = 1
    prev = None

    while cur and carry:
        s = cur.val + carry
        cur.val = s % 10
        carry = s // 10
        prev = cur
        cur = cur.next

    if carry:
        prev.next = ListNode(carry)

    # reverse back
    return reverse_list(head)

# Demo
head = build_list([1,2,3])
out = add_one(head)
print("Output:", list_to_array(out))


Output: [1, 2, 4]


## Problem 14: Given a sorted array and a target, return the index if found; otherwise insertion index

### Question
**Input:** `nums = [1, 3, 5, 6]`, `target = 5`  
**Output:** `2`

### Explanation (Approach)
Binary search for the leftmost position where target can be inserted.
If target exists, that position is its index; otherwise it's the insertion index.
Time: **O(log n)**, Space: **O(1)**.


In [15]:
def search_insert(nums: List[int], target: int) -> int:
    lo, hi = 0, len(nums)
    while lo < hi:
        mid = (lo + hi) // 2
        if nums[mid] < target:
            lo = mid + 1
        else:
            hi = mid
    return lo

# Demo
print(search_insert([1,3,5,6], 5))
print(search_insert([1,3,5,6], 2))  # insertion example


2
1


## Problem 15: Find the minimum element in a rotated sorted array

### Question
**Input:** `[4, 5, 6, 7, 0, 1, 2]`  
**Output:** `0`

### Explanation (Approach)
Use binary search:
- If `nums[mid] > nums[hi]`, min is in right half
- Else min is in left half including mid
Time: **O(log n)**.


In [16]:
def find_min_rotated(nums: List[int]) -> int:
    lo, hi = 0, len(nums) - 1
    while lo < hi:
        mid = (lo + hi) // 2
        if nums[mid] > nums[hi]:
            lo = mid + 1
        else:
            hi = mid
    return nums[lo]

# Demo
print(find_min_rotated([4,5,6,7,0,1,2]))


0


## Problem 16: Search for a target value in a rotated sorted array

### Question
**Input:** `nums = [4, 5, 6, 7, 0, 1, 2]`, `target = 0`  
**Output:** `4`

### Explanation (Approach)
Modified binary search:
At each step, one half is sorted. Decide if target lies in sorted half or other half.
Time: **O(log n)**.


In [17]:
def search_rotated(nums: List[int], target: int) -> int:
    lo, hi = 0, len(nums) - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        if nums[mid] == target:
            return mid
        # left half sorted
        if nums[lo] <= nums[mid]:
            if nums[lo] <= target < nums[mid]:
                hi = mid - 1
            else:
                lo = mid + 1
        else:
            # right half sorted
            if nums[mid] < target <= nums[hi]:
                lo = mid + 1
            else:
                hi = mid - 1
    return -1

# Demo
print(search_rotated([4,5,6,7,0,1,2], 0))


4


## Problem 17: Find the peak element in an array (peak > neighbors)

### Question
**Input:** `nums = [1, 2, 3, 1]`  
**Output:** `2` (index)

### Explanation (Approach)
Binary search on slope:
If `nums[mid] < nums[mid+1]`, peak lies to the right; else to the left (including mid).
Time: **O(log n)**.


In [18]:
def find_peak(nums: List[int]) -> int:
    lo, hi = 0, len(nums) - 1
    while lo < hi:
        mid = (lo + hi) // 2
        if nums[mid] < nums[mid + 1]:
            lo = mid + 1
        else:
            hi = mid
    return lo

# Demo
print(find_peak([1,2,3,1]))


2


## Problem 18: Count the number of negative numbers in a sorted m x n matrix

### Question
Matrix rows and columns are sorted in ascending order.  
**Input:** `[[4,3,2,-1],[3,2,1,-1],[1,1,-1,-2],[-1,-1,-2,-3]]`  
**Output:** `8`

### Explanation (Approach)
Start from top-right corner:
- If current value is negative, then all values below in that column are negative.
  Add `(m - row)` and move left.
- Else move down.
Time: **O(m+n)**.


In [19]:
def count_negatives(grid: List[List[int]]) -> int:
    if not grid or not grid[0]:
        return 0
    m, n = len(grid), len(grid[0])
    r, c = 0, n - 1
    count = 0
    while r < m and c >= 0:
        if grid[r][c] < 0:
            count += (m - r)
            c -= 1
        else:
            r += 1
    return count

# Demo
grid = [[4,3,2,-1],
        [3,2,1,-1],
        [1,1,-1,-2],
        [-1,-1,-2,-3]]
print(count_negatives(grid))


8


## Problem 19: Search a target in a 2D matrix with row-major sorted property

### Question
Each row is sorted and the first element of each row is greater than the last element of previous row.  
**Input:** `matrix = [[1,3,5,7],[10,11,16,20],[23,30,34,60]]`, `target = 3`  
**Output:** `True`

### Explanation (Approach)
Because the matrix behaves like a single sorted array of length `m*n`, do binary search with index mapping:
- `row = mid // n`
- `col = mid % n`
Time: **O(log(m*n))**.


In [20]:
def search_matrix(matrix: List[List[int]], target: int) -> bool:
    if not matrix or not matrix[0]:
        return False
    m, n = len(matrix), len(matrix[0])
    lo, hi = 0, m*n - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        r, c = divmod(mid, n)
        val = matrix[r][c]
        if val == target:
            return True
        elif val < target:
            lo = mid + 1
        else:
            hi = mid - 1
    return False

# Demo
matrix = [[1,3,5,7],[10,11,16,20],[23,30,34,60]]
print(search_matrix(matrix, 3))
print(search_matrix(matrix, 13))


True
False


## Problem 20: Find Median in Two Sorted Arrays

### Question
**Input:** `nums1 = [1, 3]`, `nums2 = [2]`  
**Output:** `2.0`

### Explanation (Approach)
Use partition-based binary search on the smaller array (classic O(log(min(m,n))) solution):
We choose `i` elements from nums1 and `j` from nums2 such that left side has half elements.
Then median is based on max-left and min-right.
Time: **O(log(min(m,n)))**.


In [21]:
def find_median_sorted_arrays(nums1: List[int], nums2: List[int]) -> float:
    # Ensure nums1 is smaller
    if len(nums1) > len(nums2):
        nums1, nums2 = nums2, nums1

    m, n = len(nums1), len(nums2)
    total_left = (m + n + 1) // 2  # elements in left partition

    lo, hi = 0, m
    while lo <= hi:
        i = (lo + hi) // 2
        j = total_left - i

        left1 = nums1[i-1] if i > 0 else -math.inf
        right1 = nums1[i] if i < m else math.inf
        left2 = nums2[j-1] if j > 0 else -math.inf
        right2 = nums2[j] if j < n else math.inf

        if left1 <= right2 and left2 <= right1:
            # correct partition
            if (m + n) % 2 == 1:
                return float(max(left1, left2))
            return (max(left1, left2) + min(right1, right2)) / 2.0
        elif left1 > right2:
            hi = i - 1
        else:
            lo = i + 1

    raise ValueError("Input arrays are not sorted properly.")

# Demo
print(find_median_sorted_arrays([1,3], [2]))
print(find_median_sorted_arrays([1,2], [3,4]))


2.0
2.5


## Problem 21: Given a sorted character array and a target letter, find the smallest letter greater than target

### Question
**Input:** `letters = ['c', 'f', 'j']`, `target = 'a'`  
**Output:** `'c'`

### Explanation (Approach)
Binary search for first letter > target.
If none found, wrap around and return the first letter.
Time: **O(log n)**.


In [22]:
def next_greatest_letter(letters: List[str], target: str) -> str:
    lo, hi = 0, len(letters)
    while lo < hi:
        mid = (lo + hi) // 2
        if letters[mid] <= target:
            lo = mid + 1
        else:
            hi = mid
    return letters[lo % len(letters)]

# Demo
print(next_greatest_letter(['c','f','j'], 'a'))
print(next_greatest_letter(['c','f','j'], 'j'))  # wrap -> 'c'


c
c


## Problem 22: Sort colors (red, white, blue) in-place

### Question
**Input:** `nums = [2, 0, 2, 1, 1, 0]`  
**Output:** `[0, 0, 1, 1, 2, 2]`

### Explanation (Approach)
Dutch National Flag algorithm:
- `low` for next 0 position
- `mid` current index
- `high` for next 2 position
Swap based on nums[mid].
Time: **O(n)**, Space: **O(1)**.


In [23]:
def sort_colors(nums: List[int]) -> None:
    low, mid, high = 0, 0, len(nums) - 1
    while mid <= high:
        if nums[mid] == 0:
            nums[low], nums[mid] = nums[mid], nums[low]
            low += 1
            mid += 1
        elif nums[mid] == 1:
            mid += 1
        else:  # 2
            nums[mid], nums[high] = nums[high], nums[mid]
            high -= 1

# Demo
nums = [2,0,2,1,1,0]
sort_colors(nums)
print(nums)


[0, 0, 1, 1, 2, 2]


## Problem 23: Find the kth largest element in an unsorted array

### Question
**Input:** `nums = [3, 2, 1, 5, 6, 4]`, `k = 2`  
**Output:** `5`

### Explanation (Approach)
Use Quickselect (average O(n)):
Partition array around a pivot so that larger elements are on one side.
Then recurse/iterate only into the relevant side.
Time: average **O(n)**, worst **O(n^2)**, Space: **O(1)** extra (in-place).


In [24]:
import random

def kth_largest(nums: List[int], k: int) -> int:
    # Convert to "kth smallest index" in descending order
    # kth largest => index k-1 in sorted(desc)
    target = k - 1

    def partition(left, right, pivot_index):
        pivot_value = nums[pivot_index]
        nums[pivot_index], nums[right] = nums[right], nums[pivot_index]
        store = left
        for i in range(left, right):
            if nums[i] > pivot_value:   # '>' for descending
                nums[store], nums[i] = nums[i], nums[store]
                store += 1
        nums[right], nums[store] = nums[store], nums[right]
        return store

    left, right = 0, len(nums) - 1
    while True:
        pivot_index = random.randint(left, right)
        pivot_index = partition(left, right, pivot_index)
        if pivot_index == target:
            return nums[pivot_index]
        elif pivot_index < target:
            left = pivot_index + 1
        else:
            right = pivot_index - 1

# Demo
print(kth_largest([3,2,1,5,6,4], 2))


5


## Problem 24: Reorder array in-place so that nums[0] <= nums[1] >= nums[2] <= nums[3] ... (wiggle)

### Question
**Input:** `nums = [3, 5, 2, 1, 6, 4]`  
**Output:** `[3, 5, 1, 6, 2, 4]` (one valid answer)

### Explanation (Approach)
One-pass greedy:
For each index i from 1..n-1:
- If i is odd, enforce `nums[i] >= nums[i-1]`
- If i is even, enforce `nums[i] <= nums[i-1]`
If violated, swap `nums[i]` and `nums[i-1]`.

This guarantees the wiggle condition.
Time: **O(n)**, Space: **O(1)**.


In [25]:
def wiggle_sort(nums: List[int]) -> None:
    for i in range(1, len(nums)):
        if (i % 2 == 1 and nums[i] < nums[i-1]) or (i % 2 == 0 and nums[i] > nums[i-1]):
            nums[i], nums[i-1] = nums[i-1], nums[i]

# Demo
nums = [3,5,2,1,6,4]
wiggle_sort(nums)
print(nums)


[3, 5, 1, 6, 2, 4]


## Problem 25: Given an array of integers, calculate the sum of all its elements

### Question
**Input:** `[1, 2, 3, 4, 5]`  
**Output:** `15`

### Explanation (Approach)
Use Python's built-in `sum` function (linear scan).
Time: **O(n)**.


In [26]:
def array_sum(nums: List[int]) -> int:
    return sum(nums)

# Demo
print(array_sum([1,2,3,4,5]))


15


## Problem 26: Find the maximum element in an array of integers

### Question
**Input:** `[3, 7, 2, 9, 4, 1]`  
**Output:** `9`

### Explanation (Approach)
Linear scan or built-in `max`.
Time: **O(n)**.


In [27]:
def find_max(nums: List[int]) -> int:
    return max(nums)

# Demo
print(find_max([3,7,2,9,4,1]))


9


## Problem 27: Implement linear search to find the index of a target element in an array

### Question
**Input:** `[5, 3, 8, 2, 7, 4]`, `target = 8`  
**Output:** `2`

### Explanation (Approach)
Scan from left to right and return the first index where value equals target.
Time: **O(n)**.


In [28]:
def linear_search(nums: List[int], target: int) -> int:
    for i, x in enumerate(nums):
        if x == target:
            return i
    return -1

# Demo
print(linear_search([5,3,8,2,7,4], 8))


2


## Problem 28: Calculate the factorial of a given number

### Question
**Input:** `5`  
**Output:** `120`

### Explanation (Approach)
Factorial: `n! = n*(n-1)*...*1` with 0! = 1.
Use iterative multiplication to avoid recursion depth issues.
Time: **O(n)**.


In [29]:
def factorial(n: int) -> int:
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    res = 1
    for i in range(2, n+1):
        res *= i
    return res

# Demo
print(factorial(5))


120


## Problem 29: Check if a given number is a prime number

### Question
**Input:** `7`  
**Output:** `True`

### Explanation (Approach)
A number `n` is prime if it has no divisors other than 1 and itself.
Check divisibility up to `sqrt(n)`.
Time: **O(sqrt(n))**.


In [30]:
def is_prime(n: int) -> bool:
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

# Demo
print(is_prime(7))
print(is_prime(9))


True
False


## Problem 30: Generate the Fibonacci series up to a given number n

### Question
The sample suggests **n terms**.  
**Input:** `8`  
**Output:** `[0, 1, 1, 2, 3, 5, 8, 13]`

### Explanation (Approach)
Generate first `n` Fibonacci numbers iteratively:
Start with a=0, b=1 and update `(a,b)=(b,a+b)`.
Time: **O(n)**.


In [31]:
def fibonacci_series(n: int) -> List[int]:
    if n <= 0:
        return []
    if n == 1:
        return [0]
    out = [0, 1]
    while len(out) < n:
        out.append(out[-1] + out[-2])
    return out

# Demo
print(fibonacci_series(8))


[0, 1, 1, 2, 3, 5, 8, 13]


## Problem 31: Calculate the power of a number using recursion

### Question
**Input:** `base = 3`, `exponent = 4`  
**Output:** `81`

### Explanation (Approach)
Use recursion (fast exponentiation):
- If exponent is 0 => 1
- If exponent is even => (base^(exp/2))^2
- If exponent is odd  => base * base^(exp-1)

Time: **O(log exponent)**.


In [32]:
def power_recursive(base: int, exponent: int) -> int:
    if exponent < 0:
        raise ValueError("This version assumes non-negative exponent")
    if exponent == 0:
        return 1
    if exponent == 1:
        return base
    half = power_recursive(base, exponent // 2)
    if exponent % 2 == 0:
        return half * half
    return base * half * half

# Demo
print(power_recursive(3, 4))


81


## Problem 32: Reverse a given string

### Question
**Input:** `"hello"`  
**Output:** `"olleh"`

### Explanation (Approach)
Use slicing `s[::-1]` which reverses a string efficiently.
Time: **O(n)**.


In [33]:
def reverse_string(s: str) -> str:
    return s[::-1]

# Demo
print(reverse_string("hello"))


olleh
